In [33]:
import pandas as pd
import numpy as np
df=pd.read_csv('Price_Agriculture_commodities_Week.csv')
df

,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min Price,Max Price,Modal Price
0,Gujarat,Amreli,Damnagar,Bhindi(Ladies Finger),Bhindi,FAQ,27-07-2023,4100.0,4500.0,4350.0
1,Gujarat,Amreli,Damnagar,Brinjal,Other,FAQ,27-07-2023,2200.0,3000.0,2450.0
2,Gujarat,Amreli,Damnagar,Cabbage,Cabbage,FAQ,27-07-2023,2350.0,3000.0,2700.0
3,Gujarat,Amreli,Damnagar,Cauliflower,Cauliflower,FAQ,27-07-2023,7000.0,7500.0,7250.0
4,Gujarat,Amreli,Damnagar,Coriander(Leaves),Coriander,FAQ,27-07-2023,8400.0,9000.0,8850.0
...,...,...,...,...,...,...,...,...,...,...
23088,Uttrakhand,Haridwar,Roorkee,Mango,Other,Medium,02-08-2023,800.0,1200.0,1000.0
23089,Uttrakhand,Haridwar,Roorkee,Mousambi(Sweet Lime),Other,Medium,02-08-2023,1500.0,2500.0,2000.0
23090,Uttrakhand,Haridwar,Roorkee,Pear(Marasebu),Other,Medium,02-08-2023,2000.0,3000.0,2500.0
23091,Uttrakhand,Haridwar,Roorkee,Potato,Other,FAQ,02-08-2023,900.0,1800.0,1500.0


In [34]:
df['Commodity'].unique().shape

(234,)

In [35]:
df.isnull().sum()

State           0
District        0
Market          0
Commodity       0
Variety         0
Grade           0
Arrival_Date    0
Min Price       0
Max Price       0
Modal Price     0
dtype: int64

In [36]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
label_encoders = {}

# Iterate over each column and encode its values
for col in df.columns:
    if df[col].dtype == 'object':  # Check if the column is categorical
        label_encoders[col] = LabelEncoder()  # Initialize LabelEncoder
        df[col] = label_encoders[col].fit_transform(df[col])  # Fit and transform the column
# Iterate over each column and perform inverse transformation
# for col in label_encoders:
#     decoded_df[col] = label_encoders[col].inverse_transform(decoded_df[col])


In [37]:
df.corr()

,State,District,Market,Commodity,Variety,Grade,Arrival_Date,Min Price,Max Price,Modal Price
State,1.000000,-0.105156,-0.074029,0.092157,-0.153660,-0.007290,0.001506,-0.073047,-0.123196,-0.103140
District,-0.105156,1.000000,0.389171,-0.003934,0.030322,0.005608,-0.009432,0.024767,0.033663,0.029444
Market,-0.074029,0.389171,1.000000,-0.009142,0.014995,0.003537,0.008418,0.048077,0.048210,0.048116
Commodity,0.092157,-0.003934,-0.009142,1.000000,0.374992,-0.050282,-0.005132,-0.097973,-0.100830,-0.100769
Variety,-0.153660,0.030322,0.014995,0.374992,1.000000,0.049990,-0.024254,0.006845,0.021528,0.015287
Grade,-0.007290,0.005608,0.003537,-0.050282,0.049990,1.000000,0.006541,0.017998,0.030275,0.023141
Arrival_Date,0.001506,-0.009432,0.008418,-0.005132,-0.024254,0.006541,1.000000,-0.014817,-0.025398,-0.021186
Min Price,-0.073047,0.024767,0.048077,-0.097973,0.006845,0.017998,-0.014817,1.000000,0.954383,0.985556
Max Price,-0.123196,0.033663,0.048210,-0.100830,0.021528,0.030275,-0.025398,0.954383,1.000000,0.982914
Modal Price,-0.103140,0.029444,0.048116,-0.100769,0.015287,0.023141,-0.021186,0.985556,0.982914,1.000000


In [38]:
from sklearn.model_selection import train_test_split
X=df.drop(columns='Commodity')
y=df['Commodity']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=420)


In [40]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier()
tree.fit(X_train,y_train)
y_pred=tree.predict(X_test)
print(accuracy_score(y_pred,y_test))


0.696397644613786


In [42]:
# 3. Decision Tree Classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, randint
from sklearn.preprocessing import StandardScaler
param_dist = {
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': randint(1, 1000),  # Adjust the range based on your data and computational resources
    'classifier__min_samples_split': randint(2, 20),
    'classifier__min_samples_leaf': randint(1, 20),
    'classifier__max_features': ['sqrt', 'log2', None],
    'classifier__class_weight': [None, 'balanced']
}
DTC=DecisionTreeClassifier()
pipeline=Pipeline([('preprocessor',StandardScaler()),('classifier',DTC)])
random_search=RandomizedSearchCV(estimator=pipeline,param_distributions=param_dist,cv=5,n_iter=10,scoring='accuracy')
random_search.fit(X_train,y_train)
best_param2=random_search.best_params_
best_p2=Pipeline([('preprocessor',StandardScaler()),('classifier',DecisionTreeClassifier())])
best_p2.set_params(**best_param2)
best_p2.fit(X_train,y_train)

y_pred=best_p2.predict(X_test)
accuracy3 = accuracy_score(y_test, y_pred)
print("Accuracy Score of DecisionTreeClassifier: %.3f"%accuracy3)


c:\Users\91620\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Accuracy Score of DecisionTreeClassifier: 0.589
